Dependencies and packages

In [29]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from sklearn.cluster import KMeans

Import Data

In [2]:
ddb = gpd.read_file('./data/ddb_roundabout.geojson')

In [16]:
accident = gpd.read_file('./data/crash_roundabout.geojson')

In [31]:
roundabout = gpd.read_file('./data/roundabout.geojson')

In [32]:
roundabout.describe()

,id,@id,access,area,bicycle,bridge,bridge_number,check_date,construction,cycleway,...,source:geometry,source:lanes,source:name,source:oneway,surface,traffic_calming,turn:lanes,width,wikidata,geometry
count,904,904,9,4,11,4,3,1,1,1,...,463,521,5,8,884,4,15,12,13,904
unique,904,904,3,1,2,1,2,1,1,1,...,11,4,2,1,4,1,5,10,1,904
top,way/1059094803,way/1059094803,private,yes,no,yes,3075,2022-05-12,residential,lane,...,ACT2018,ACTmapi Imagery 2018,LPI NSW Base Map,ACTmapi Imagery 2018,asphalt,island,|right,6,Q3365467,"POLYGON ((149.0275191 -35.2024024, 149.0275175..."
freq,1,1,6,4,6,4,2,1,1,1,...,151,415,4,8,475,4,10,3,13,1


steps: (option 1)

The goal is to create a graph similar to version 1 but with lesser edges and lesser nodes for a clearer defined graph

1. cluster rows for ddb dataset (based on location)
2. group or cluster their x acc, y acc, and classification to the clustered rows
3. create empty graph
4. add nodes based on clustered rows (location) dataset
5. add edges based on relationship connected clustered rows (location)

In [60]:
#Step 1 - cluser rows for location only
locations = [[point.x, point.y] for point in ddb['geometry']]
locations = np.array(locations)
locations

array([[16595825.83481508, -4208110.69446205],
       [16597539.70969533, -4206559.21011938],
       [16597539.70969533, -4206559.21011938],
       ...,
       [16605925.40693679, -4207808.0751288 ],
       [16606322.59487994, -4208557.81676017],
       [16605921.17679614, -4207828.81342951]])

In [61]:
np.info(locations)

class:  ndarray
shape:  (2167, 2)
strides:  (16, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x22b24ca21e0
byteorder:  little
byteswap:  False
type: float64


In [40]:
num_location_clusters = 400

In [63]:
kmeans_location = KMeans(n_clusters=num_location_clusters)

In [68]:
ddb['location_cluster'] = kmeans_location.fit(locations)

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [71]:
#Step 2 Group/cluster Xacc, Yacc, Classification
# Group or cluster 'x-acceleration' within the location clusters
ddb['x_acc_cluster'] = ddb.groupby('location_cluster')['xacc'].transform('mean')

# Group or cluster 'y-acceleration' within the location clusters
ddb['y_acc_cluster'] = ddb.groupby('location_cluster')['yacc'].transform('mean')

# Group or cluster 'classification' within the location clusters (you can use other aggregation functions)
ddb['classification_cluster'] = ddb.groupby('location_cluster')['classification'].transform('mean')

TypeError: Could not convert BrakingBrakingBrakingSteeringSteeringSteeringSteeringSteeringSteeringBrakingBrakingSteeringBrakingSteeringSteeringBrakingBrakingBrakingBrakingSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringBrakingSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringBrakingBrakingBrakingBrakingSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringBrakingBrakingBrakingSteeringSteeringBrakingSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringBrakingBrakingSteeringBrakingSteeringSteeringSteeringSteeringSteeringBrakingSteeringSteeringSteeringBrakingSteeringSteeringBrakingSteeringBrakingSteeringSteeringBrakingSteeringSteeringSteeringBrakingSteeringSteeringSteeringSteeringBrakingSteeringBrakingBrakingSteeringBrakingSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringSteeringBrakingSteeringSteeringSteeringBrak

In [67]:
#Step 3 Create empty graph
G = nx.Graph()

In [ ]:
#Step 4 Add nodes to graph based on location clusters
for cluster_label in ddb['location_cluster'].unique():
    G.add_node(f'LocationCluster{cluster_label}')

In [ ]:
#Step 5

In [ ]:
#Step 6 - Show Graph

steps: (option 2)
1. 